In [1]:
#PANDAS
import pandas as pd

def actors_and_directors(actor_director: pd.DataFrame) -> pd.DataFrame:
    out = (
        actor_director
        .groupby(['actor_id', 'director_id'])
        .size()
        .reset_index(name='cnt')
    )

    return out.loc[out['cnt'] >= 3, ['actor_id', 'director_id']]


#SQL

SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3;

In [1]:
#PANDAS

import pandas as pd

def fix_names(users: pd.DataFrame) -> pd.DataFrame:
    out = users.assign(name=users['name'].str.capitalize()).sort_values('user_id')
    return out

#SQL

SELECT user_id,
       CONCAT(UPPER(LEFT(name, 1)), LOWER(SUBSTRING(name, 2))) AS name
FROM Users
ORDER BY user_id;

In [ ]:
#PANDAS

import pandas as pd

def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    out = (person.merge(address, how='left', on='personId')[['firstName','lastName','city','state']])

    return out

#SQL

SELECT p.firstName, p.lastName, a.city, a.state
FROM Person p
LEFT JOIN Address a
  ON p.personId = a.personId;

In [ ]:
#PANDAS

import pandas as pd

def second_highest_salary(employee: pd.DataFrame) -> pd.DataFrame:
    s = (employee['salary'].dropna().drop_duplicates().sort_values(ascending=False).reset_index(drop=True))
    out = pd.DataFrame({'SecondHighestSalary': [s.iloc[1] if len(s) > 1 else None]})

    return out

#SQL

SELECT (
  SELECT DISTINCT salary
  FROM Employee
  ORDER BY salary DESC
  LIMIT 1 OFFSET 1
) AS SecondHighestSalary;

In [ ]:
#PANDAS

import pandas as pd

def list_products(products: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    feb = (orders['order_date'] >= pd.Timestamp('2020-02-01')) & (orders['order_date'] < pd.Timestamp('2020-03-01'))
    feb_orders = orders.loc[feb]

    sums = (feb_orders.groupby('product_id', as_index=False)['unit'].sum())

    out = (sums.merge(products[['product_id', 'product_name']], on='product_id', how='left').loc[lambda d: d['unit'] >= 100, ['product_name', 'unit']])

    return out

#SQL

SELECT
  p.product_name,
  SUM(o.unit) AS unit
FROM Products AS p
JOIN Orders   AS o
  ON p.product_id = o.product_id
WHERE o.order_date >= '2020-02-01'
  AND o.order_date <  '2020-03-01'   -- Feb 2020 only
GROUP BY p.product_id, p.product_name
HAVING SUM(o.unit) >= 100;

In [ ]:
#PANDAS

import pandas as pd

def replace_employee_id(employees: pd.DataFrame, employee_uni: pd.DataFrame) -> pd.DataFrame:
    out = employees.merge(employee_uni, how='left', on='id')[['unique_id','name']]
    return out

#SQL

SELECT eu.unique_id, e.name
FROM Employees e
LEFT JOIN EmployeeUNI eu
  ON e.id = eu.id;

In [ ]:
#PANDAS

import pandas as pd

def gameplay_analysis(activity: pd.DataFrame) -> pd.DataFrame:
    firsts = (activity.groupby('player_id', as_index=False)['event_date'].min().rename(columns={'event_date':'first_login'}))

    next_day = activity[['player_id','event_date']].copy()
    next_day['event_date'] = next_day['event_date'] - pd.Timedelta(days=1)

    joined = firsts.merge(next_day, left_on=['player_id','first_login'],right_on=['player_id','event_date'], how='left')

    fraction = round(joined['event_date'].notna().mean(), 2)
    out = pd.DataFrame({'fraction': [fraction]})
    return out

#SQL

WITH firsts AS (
  SELECT player_id, MIN(event_date) AS first_login
  FROM Activity
  GROUP BY player_id
)
SELECT ROUND(
  AVG(CASE WHEN EXISTS (
        SELECT 1
        FROM Activity a2
        WHERE a2.player_id = f.player_id
          AND a2.event_date = DATE_ADD(f.first_login, INTERVAL 1 DAY)
  ) THEN 1 ELSE 0 END), 2
) AS fraction
FROM firsts f;

In [ ]:
#PANDAS

import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    out = (project.merge(employee, on='employee_id').groupby('project_id', as_index=False)['experience_years'].mean().rename(columns={'experience_years':'average_years'}))
    out['average_years'] = out['average_years'].round(2)
    return out

#SQL

SELECT p.project_id,
       ROUND(AVG(e.experience_years), 2) AS average_years
FROM Project p
JOIN Employee e
  ON p.employee_id = e.employee_id
GROUP BY p.project_id;

In [ ]:
#PANDAS

import pandas as pd

def top_three_salaries(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    df = employee.merge(department, left_on='departmentId', right_on='id',suffixes=('_emp','_dept'))
    df['rnk'] = (df.groupby('departmentId')['salary'].rank(method='dense', ascending=False))
    out = df.loc[df['rnk'] <= 3, ['name_dept','name_emp','salary']].rename(columns={'name_dept':'Department','name_emp':'Employee'})
    return out

#SQL

WITH ranked AS (
  SELECT d.name AS Department,
         e.name AS Employee,
         e.salary,
         DENSE_RANK() OVER (PARTITION BY d.id ORDER BY e.salary DESC) AS rnk
  FROM Employee e
  JOIN Department d ON e.departmentId = d.id
)
SELECT Department, Employee, salary
FROM ranked
WHERE rnk <= 3;